### Optimization - QCQP, barrier method

##### 1.다음과 같은 QCQP 형태의 최적화 문제를 고려.
##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx $, 
##### $\text{s.t} \quad \frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0  \quad i = 1, \dots , m$

##### 위의 부등제약 조건을 아래와 같이 지시함수 형태로 포함, 여전히 boundary를 가지고 잇고, 미분 불가능 하기 때문에 계산상 어려움 존재 
##### $C = \{x:\frac{1}{2}x^TQ_ix + p_{i}^{T}x \leq 0 \quad i = 1, \dots , m\}$

##### $\text{minimize}_x \quad \frac{1}{2}x^TQx + p^Tx + I_c(x)$ 
##### 해결) barrier function을 이용해 boundary 포함하지않고, 미분가능 하게해서 newton's method 적용

##### 2.logarithmic barrier function 구하기
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x 라고 하자.$ -> convex이고, 두번 미분가능
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ -> logarithmic barrier function

##### 3.barrier function을 적용 시킨 최적화 문제.
##### $\text{minimize}_x \quad t(\frac{1}{2}x^TQx + p^Tx) + \phi(x)$ 
##### 이와 같이 근사해서 정의된 문제를 newton's method로 푸는 방법을 barrier method라고 함.

##### 4.log barrier 계산.
##### $h_i(x)=\frac{1}{2}x^TQ_ix + p_{i}^{T}x$ 
##### $\phi(x)= -\sum_{i=1}^{m}log(-h_i(x))$ 
##### gradient: $\bigtriangledown \phi(x) = - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown h_i(x)$
##### Hessian: $\bigtriangledown^2 \phi(x) = \sum_{i=1}^{m} \frac{1}{h_i(x)^2}\bigtriangledown h_i(x)\bigtriangledown h_i(x)^T - \sum_{i=1}^{m} \frac{1}{h_i(x)}\bigtriangledown^2 h_i(x) $

In [25]:
import numpy as np
import tensorflow as tf
#data generation
n = 3

np.random.seed(1)

Q1_ = np.random.random(size=(n,n))
Q1 = tf.Variable(Q1_.T * Q1_)
Q1 = tf.reshape(Q1, [1,3,3])
Q2_ = np.random.random(size=(n,n))
Q2 = tf.Variable(Q2_.T * Q2_)
Q2 = tf.reshape(Q2, [1,3,3])
Q3_ = np.random.random(size=(n,n))
Q3 = tf.Variable(Q3_.T * Q3_)
Q3 = tf.reshape(Q3, [1,3,3])
Q0_ = np.random.random(size=(n,n))
Q0 = tf.Variable(Q0_.T * Q0_)
Q0 = tf.reshape(Q0, [1,3,3])

Q = tf.concat([Q0, Q1, Q2, Q3], axis=0)

c1 = tf.Variable(np.random.random(size=(n,1)))
c1 = tf.reshape(c1, [1,3,1])
c2 = tf.Variable(np.random.random(size=(n,1)))
c2 = tf.reshape(c2, [1,3,1])
c3 = tf.Variable(np.random.random(size=(n,1)))
c3 = tf.reshape(c3, [1,3,1])
c0 = tf.Variable(np.random.random(size=(n,1)))
c0 = tf.reshape(c0, [1,3,1])

c = tf.concat([c0, c1, c2, c3], axis=0)

x = tf.Variable(-np.random.random(size=(n,1)))

t = tf.constant(1)
#%% objective function
def h(Q,c,x):
    return tf.matmul(tf.transpose(c), x) + tf.matmul(tf.matmul(tf.transpose(x), Q), x)/2

def obj_fn(Q, c, x, t):
    return t.numpy()*h(Q[0], c[0], x) -tf.reduce_sum(tf.math.log([-h(Q[1], c[1], x), -h(Q[2], c[2], x), -h(Q[3], c[3], x)]))
#%% iteration
for i in range(10):
    print("Iteration", i, "\n x :\n", x.numpy(), "\nobj :", h(Q[0], c[0], x).numpy())
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch(x)
            loss = obj_fn(Q, c, x, t)
        grads = tape1.gradient(loss, x)
    hessian = tf.reshape(tape2.jacobian(grads, x), shape=(3,3))
    x = tf.Variable(x - tf.matmul(tf.linalg.inv(hessian), grads)) # x 업데이트


Iteration 0 
 x :
 [[-0.28777534]
 [-0.13002857]
 [-0.01936696]] 
obj : [[-0.24623822]]
Iteration 1 
 x :
 [[-0.05790141]
 [-0.53790817]
 [-0.14390974]] 
obj : [[-0.47451565]]
Iteration 2 
 x :
 [[ 0.47679513]
 [-1.35535807]
 [-0.38672035]] 
obj : [[-0.71071905]]
Iteration 3 
 x :
 [[ 1.05098917]
 [-2.03842893]
 [-0.93986394]] 
obj : [[-0.47264719]]
Iteration 4 
 x :
 [[ 1.13909119]
 [-2.10932333]
 [-1.10364447]] 
obj : [[-0.36016049]]
Iteration 5 
 x :
 [[ 1.13901824]
 [-2.10903491]
 [-1.11181366]] 
obj : [[-0.35464009]]
Iteration 6 
 x :
 [[ 1.1389439 ]
 [-2.10898701]
 [-1.11181281]] 
obj : [[-0.35464155]]
Iteration 7 
 x :
 [[ 1.13894389]
 [-2.10898701]
 [-1.11181281]] 
obj : [[-0.35464155]]
Iteration 8 
 x :
 [[ 1.13894389]
 [-2.10898701]
 [-1.11181281]] 
obj : [[-0.35464155]]
Iteration 9 
 x :
 [[ 1.13894389]
 [-2.10898701]
 [-1.11181281]] 
obj : [[-0.35464155]]
